In [1]:
using VCFTool

┌ Info: Precompiling VCFTool [8f8d80d9-e796-45a3-8ef2-c0461a3110a4]
└ @ Base loading.jl:1278


In [2]:
# For viewing DataFrames

ENV["COLUMNS"]=100

ENV["LINES"]=200

# For viewing JuliaDB tables and arrays

using IndexedTables

IndexedTables.set_show_compact!(false)

false

In [3]:
# JuliaDB.tracktime(f)

In [4]:
input_dir = "../input/"

vcf_738_file_path = joinpath(input_dir, "738_variants.vcf.gz")

vcf_738_1000_file_path = joinpath(input_dir, "738_variants_1000.vcf.gz")

vcf_769_3_file_path = joinpath(input_dir, "769_3_variants.vcf.gz")

vcf_test_file_path = joinpath(input_dir, "test.vcf.gz")

vcf_file_path_to_use = vcf_test_file_path

"../input/test.vcf.gz"

## Search by Variant

In [5]:
vcf = make_vcf_ndsparse(vcf_file_path_to_use)

1-d NDSparse with 10 values (10 field named tuples):
1  │ CHROM   POS      ID   REF   ALT  QUAL   FILTER                                INFO                                      FORMAT                                  GERM
───┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
2  │ "chr1"  "10144"  "."  "T"   "C"  "0"    "LowGQX;NoPassedVariantGTs"           "SNVHPOL=4;MQ=5"                          "GT:GQ:GQX:DP:DPF:AD:ADF:ADR:SB:FT:PL"  "0/1:16:0:3:0:2,1:2,1:0,0:0.0:LowGQX:19,0,52"
3  │ "chr1"  "10296"  "."  "C"   "G"  "0"    "LowGQX;NoPassedVariantGTs"           "SNVHPOL=4;MQ=9"                          "GT:GQ:GQX:DP:DPF:AD:ADF:ADR:SB:FT:PL"  "0/1:22:0:3:9:2,1:0,1:2,0:0.0:LowGQX:25,0,33"
4  │ "chr1"  "10327"  "."  "T"   "C"  "0"    "LowGQX;NoPassedVariantGTs"           "SNVHPOL=4;MQ=11"                         "GT:GQ:GQX:D

In [6]:
# Convert chromosome strings to chromosome numbers

chromosome_string = columns(vcf)[2]

chromosome_numeric = []

for item in chromosome_string

    parts = split(item, "")

    chromosome_number = parts[length(parts)]

    push!(chromosome_numeric, parse(Int, chromosome_number))

end

10-element Array{Any,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [9]:
# using IndexedTables

# vcf_new = IndexedTables.reindex(vcf, (:CHROM, :POS))

10-element StructArray(::WeakRefStrings.StringArray{String,1}, ::WeakRefStrings.StringArray{String,1}, ::WeakRefStrings.StringArray{String,1}, ::WeakRefStrings.StringArray{String,1}, ::WeakRefStrings.StringArray{String,1}, ::WeakRefStrings.StringArray{String,1}, ::WeakRefStrings.StringArray{String,1}, ::WeakRefStrings.StringArray{String,1}, ::WeakRefStrings.StringArray{String,1}, ::WeakRefStrings.StringArray{String,1}) with eltype NamedTuple{(:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM),NTuple{10,String}}:
 (CHROM = "chr1", POS = "10144", ID = ".", REF = "T", ALT = "C", QUAL = "0", FILTER = "LowGQX;NoPassedVariantGTs", INFO = "SNVHPOL=4;MQ=5", FORMAT = "GT:GQ:GQX:DP:DPF:AD:ADF:ADR:SB:FT:PL", GERM = "0/1:16:0:3:0:2,1:2,1:0,0:0.0:LowGQX:19,0,52")
 (CHROM = "chr1", POS = "10296", ID = ".", REF = "C", ALT = "G", QUAL = "0", FILTER = "LowGQX;NoPassedVariantGTs", INFO = "SNVHPOL=4;MQ=9", FORMAT = "GT:GQ:GQX:DP:DPF:AD:ADF:ADR:SB:FT:PL", GERM = "0/1:22:0:3:9:2,1:0,1:2,

In [ ]:
rows(vcf)

n_rows = length(rows(vcf))

vcf_new = vcf[1:end .!=1]

# keys(vcf)

# IndexedTables.where()

# Base.getindex()

# Base.filter()
# filter(val -> val == "A", vcf.:ALT)

# IndexedTables.groupby()

In [ ]:
# cols = columns(vcf_table)

# new_cols = keys(cols)[2:end]

# new_vals = values(vcf_table)[1]

d = ColDict(vcf_table)

pop!()

# getindex(vcf_table, [:POS, :ID])

# new_tuple = NamedTuple()

In [ ]:
vcf_table = loadtable(
    "../input/test.vcf",
    delim='\t',
    header_exists=false, 
    colnames=[:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM],
    indexcols=[:CHROM],
)

In [ ]:
using JuliaDB 

function make_vcf_indexedtable(vcf_gz_file_path)
    
    vcf_file_path = string(Array(split(vcf_gz_file_path, ".gz"))[1])

    if isfile(vcf_file_path) == false
        
        io_gz = open(vcf_gz_file_path)

        io = GzipDecompressorStream(io_gz)

        file = CSV.File(io, comment="##", delim='\t', header=1)

        CSV.write(vcf_file_path, delim='\t', file)

        close(io)

    end
    
    vcf_table = loadtable(
        vcf_file_path,
        delim='\t',
        header_exists=false, 
        colnames=[:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM],
        indexcols=[:CHROM],
    )

    # Remove header row
    
    n_rows = length(rows(vcf_table))

    vcf_table = vcf_table[2:n_rows]
    
    # Convert chromosome strings to chromosome numbers

    chromosomes_string = Array(columns(vcf_table)[1])

    chromosomes_numeric = []

    for item in chromosomes_string

        parts = split(item, "")

        chromosome_number = parts[length(parts)]

        push!(chromosomes_numeric, parse(Int, chromosome_number))

    end
end

In [ ]:
# https://www.snpedia.com/index.php/Rs1042522
rs1042522 = ["chr17", 7676154]

# TP53
tp53 = ["chr17", 7661779:7687550]

# test set
t = ["chr1", 10000:12000]

In [ ]:
using Dates

start_time = now()

for value in vcf["chr17", 7661779]
    
    genotype = []
    
    allele_ref = value[:REF]
    
    allele_alt = value[:ALT]
    
    numeric_genotype = split(split(value[:GERM], ":")[1], "/")
    
    if occursin("|", numeric_genotype[1])
        
        numeric_genotype = split(numeric_genotype[1], "|")
        
    end
    
    counter = 1
    
        while counter < 3
        
            if "0" == numeric_genotype[counter]

                push!(genotype, allele_ref)

            elseif "1" == numeric_genotype[counter]

                push!(genotype, split(allele_alt, ",")[1])

            elseif "2" == numeric_genotype[counter]

                push!(genotype, split(allele_alt, ",")[2])
        
            end    
    
        counter += 1
        
        end

    println("Genotype: $genotype")
    
    println("\n")
    
end
  
end_time = now()

println("\nTook $(canonicalize(Dates.CompoundPeriod(end_time - start_time))).\n")

In [ ]:
import Pkg; Pkg.add("Dagger")

using Dagger

Dagger.save(vcf, "../input/test_vcf_ndsparse_save.vcf")

In [ ]:
using Dagger

dagger_load_vcf = Dagger.load("../input/test_vcf_ndsparse_save.vcf")

## Search by Gene

## Search by Region

## Tabix Regions

In [ ]:
bed_file_path = joinpath(input_dir, "cardiotoxicity.bed")

tabix_regions_from_file(
    bed_file_path,
    vcf_738_file_path,
    output_dir,
)

## JuliaDB

An unzipped VCF file can be loaded into an indextable or ndsparse array directly using loadtable() or ndsparese() respectively. However, its much faster to query variants if they are in an ndsparse array because the chromosome and position are both part of the index.

In [ ]:
vcf_ndsparse_738 = make_ndsparse("../input/738_variants.vcf.gz")

In [ ]:
for item in vcf_ndsparse_738
    
    println(item) 

    println(item[:REF])

    end
    
end